In [22]:
import torch
import torch.nn as nn
import numpy as np
from nettest import *

Neural network implementation to model the mathematical statement, $a^2 + b^2 \approx 1$ in the context $a, b \in {\rm I\!R}$ to a predetermined precision in the training set.  

### Network Implementation

In [23]:
class Neural_Network(nn.Module):
    def __init__(self, inputSize, outputSize, hiddenSize):
        super(Neural_Network, self).__init__()
        # parameters
        # TODO: parameters can be parameterized instead of declaring them here
        self.inputSize = inputSize
        self.outputSize = outputSize
        self.hiddenSize = hiddenSize
        
        # weights
        self.W1 = torch.randn(self.inputSize, self.hiddenSize) # 2 X 3 tensor
        self.W2 = torch.randn(self.hiddenSize, self.outputSize) # 3 X 1 tensor
        self.b1 = torch.randn(self.inputSize, 1)
        self.b1 = torch.randn(self.hiddenSize, 1)
        
        
    def forward(self, X):
        self.z = torch.matmul(X, self.W1) + self.b1 # 3 X 3 ".dot" does not broadcast in PyTorch
        self.z2 = self.sigmoid(self.z) # activation function
        self.z3 = torch.matmul(self.z2, self.W2) + self.b1
        o = self.sigmoid(self.z3) # final activation function
        return o
        
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        # derivative of sigmoid
        return s * (1 - s)
    
    def backward(self, X, y, o):
        self.o_error = y - o # error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # derivative of sig to error
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        self.W1 += torch.matmul(torch.t(X), self.z2_delta)
        self.b1 += self.z2_delta
        self.W2 += torch.matmul(torch.t(self.z2), self.o_delta)
        self.b2 += self.o_delta
        
    def train(self, X, y):
        # forward + backward pass for training
        o = self.forward(X)
        self.backward(X, y, o)
        
    def saveWeights(self, model):
        # we will use the PyTorch internal storage functions
        torch.save(model, "NN")
        # you can reload model with all the weights and so forth with:
        # torch.load("NN")
        
    def predict(self):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(xPredicted))
        print ("Output: \n" + str(self.forward(xPredicted)))

### Acquiring Training Data

In [24]:
X, y = generateData(2000, 0.01)
X = torch.from_numpy(X)
y = torch.from_numpy(y)

(2000, 2) (2000, 1)


### Training the Network

In [25]:
NN = Neural_Network(2, 1, 3)
for i in range(1000):  # trains the NN 1,000 times
    print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(X))**2).detach().item()))  # mean sum squared loss
    NN.train(X, y)
NN.saveWeights(NN)
NN.predict()

RuntimeError: Expected object of scalar type Double but got scalar type Float for argument #2 'mat2' in call to _th_mm